In [1]:
# 不太想用Chrome浏览器，这里还是用MicroSoft Edge浏览器来操作
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

# 一、 利用Selenium调用Edge浏览器
driver_path = "F:/Python/msedgedriver.exe"
driver = webdriver.Edge(executable_path=driver_path) # 指定driver路径

# 访问前程无忧
driver.get("https://search.51job.com") # 浏览器打开页面

wait = WebDriverWait(driver, 10) # 显式等待10秒

before = driver.current_window_handle # 记录当前窗口

In [2]:
# 二、 搜索近三天发布的数据分析职位

# 第一步 要设置地点 南京
# addr_btn = driver.find_element_by_css_selector(
#     '#work_position_input'
# )
addr_btn = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#work_position_input')
))
addr_btn.click() # 点击地点
# hb_span = driver.find_element_by_css_selector( # 由于我现在所在城市太小没有所需职务，所以先把我的城市给去掉（其实留着也不碍事）
#     '#work_position_click_multiple_selected_each_151700'
# )
hb_span = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#work_position_click_multiple_selected_each_151700')
))
hb_span.click() # 取消我的当前城市 

# nj_span = driver.find_element_by_css_selector( # 选择南京标签
#     '#work_position_click_center_right_list_category_000000_070200'
# )
# nj_span = wait.until(EC.element_to_be_clickable(
#     (By.CSS_SELECTOR, '#work_position_click_center_right_list_category_000000_070200')
# ))
nj_span = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#work_position_click_center_right_list_category_000000_070200')
))
nj_span.click() # 选择南京
hb_span.click() # 取消我的当前城市 搞不懂为什么第一遍点击的时候没有被取消掉，第二遍点击后就被取消掉了（这是个Bug）
# addr_ok = driver.find_element_by_css_selector( # 地址选择完毕的确定按钮
#     '#work_position_click_bottom_save'
# )
addr_ok = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#work_position_click_bottom_save')
))
addr_ok.click() # 确定

In [3]:
# 第二步 选择行业
industry_btn = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#indtype_input')
)) # 行业
industry_btn.click()

IT_software = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#indtype_click_center_right_list_category_01_01')
)) # 计算机软件
IT_software.click() # 点击计算机软件行业

indu_ok =  wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#indtype_click_bottom_save')
)) 
indu_ok.click() # 选择好行业

In [4]:
# 第三步 选择职能
job_btn = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#funtype_input')
)) # 职能
job_btn.click()

data_span = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#funtype_click_center_right_list_category_0100_7500')
)) # 数据标签
data_span.click()

data_analyst = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#funtype_click_center_right_list_sub_category_each_0100_7501')
)) # 数据分析师
data_analyst.click()

data_analysis_manager = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#funtype_click_center_right_list_sub_category_each_0100_7502')
)) # 数据分析经理
data_analysis_manager.click()

job_ok =  wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#funtype_click_bottom_save')
)) 
job_ok.click() # 选择好职能

In [5]:
# 点击搜索
search_btn =  wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, 'body > div.dw_wp > form > div > div.dw_search_in > button')
)) 
search_btn.click() # 丢按保护搜索

In [6]:
# 点击近三天 页面自动筛选出近三天的职位
near_3day =  wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#filter_issuedate > ul > li:nth-of-type(3) > a')
)) 
near_3day.click() # 选择近三天

In [7]:
# 三、 获取前5页招聘数据相关信息（职位名、详细链接、公司名、工作地点、薪资、发布时间）

# 获取下一页按钮
# next_page = wait.until(EC.element_to_be_clickable(
#     (By.CSS_SELECTOR, '#resultList > div.dw_page > div > div > div > ul > li:nth-of-type(8) > a')
# )) 
next_page = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#rtNext')
))
from bs4 import BeautifulSoup

job_names = [] # 职位名
job_urls = [] # 详细链接
companies = [] # 公司名
addresses = [] # 工作地点
salaries = [] # 薪资
release_times = [] # 发布时间

# 获取前五页的数据
for j in range(1, 6):
    # 由于点击了下一页，虽然还是在当前页，但是网页的内容变了，就要重写解析页面
    driver.switch_to.window(driver.window_handles[0]) # driver.window_handles[1] 表示切换到第二个标签页上
    html = driver.page_source # 这时page_source就是新的页面的源代码了
    soup = BeautifulSoup(html, 'lxml')
    
    job_name = [i['title'] for i in soup.select('.el > p.t1 > span > a')]
    job_url = [i['href'] for i in soup.select('.el > p.t1 > span > a')]
    company = [i.text for i in soup.select('.el > span.t2 > a')]
    address = [i.text for i in soup.select('.el > span.t3')[1:]] # 因为职位标题上的标签类名与职位上的类名重复了
    salary = [i.text for i in soup.select('.el > span.t4')[1:]] # 所以匹配到的第个数据要给去掉
    release_time = [i.text for i in soup.select('.el > span.t5')[1:]] # 这三个第一个数据都是标题，这里不需要标题
    
    print(len(job_name), len(job_url), len(company), len(address), len(salary), len(release_time))
    
    job_names.extend(job_name) # 直接在原列表后追加新的数据就行
    job_urls.extend(job_url) # 这样方便直接用pandas操作
    companies.extend(company)
    addresses.extend(address)
    salaries.extend(salary)
    release_times.extend(release_time)
    print(j, "页OK！")
    # 最后再点击进入下一页
    next_page = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '#rtNext')
    )) 
    next_page.click()

50 50 50 50 50 50
1 页OK！
50 50 50 50 50 50
2 页OK！
50 50 50 50 50 50
3 页OK！
50 50 50 50 50 50
4 页OK！
50 50 50 50 50 50
5 页OK！


In [8]:
import pandas as pd
print(len(job_names), len(job_urls), len(companies), len(addresses), len(salaries), len(release_times))
data = pd.DataFrame({
    "职位名": job_names, 
    "详细链接": job_urls, 
    "公司名": companies, 
    "工作地点": addresses, 
    "薪资": salaries,
    "发布时间": release_times})
data

250 250 250 250 250 250


,职位名,详细链接,公司名,工作地点,薪资,发布时间
0,大数据分析项目经理,https://jobs.51job.com/nanjing-pkq/123452448.h...,南京拾柴信息科技有限公司,南京-浦口区,0.6-1万/月,07-10
1,数据分析师,https://jobs.51job.com/nanjing-qxq/109499250.h...,江苏东交智控科技集团股份有限公司...,南京-栖霞区,10-15万/年,07-10
2,宁波地区金融营销CRM大数据分析员，数据架构师,https://jobs.51job.com/shanghai-xhq/117343834....,上海天正软件有限公司,异地招聘,1.5-2万/月,07-10
3,数据分析师,https://jobs.51job.com/nanjing-yhtq/118070931....,新智道枢（上海）科技有限公司,南京-雨花台区,1-2万/月,07-10
4,数据分析,https://jobs.51job.com/dalian-zsq/116420034.ht...,广州华钧软件科技有限公司,异地招聘,1.5-2万/月,07-10
...,...,...,...,...,...,...
245,产品经理助理,https://jobs.51job.com/nanjing-jyq/119970248.h...,北京会畅教育科技有限公司,南京-建邺区,0.9-1.5万/月,07-10
246,高级产品经理,https://jobs.51job.com/nanjing-pkq/106645449.h...,南京安元科技有限公司,南京-浦口区,2-3万/月,07-10
247,活动策划,https://jobs.51job.com/dalian-zsq/116420167.ht...,广州华钧软件科技有限公司,异地招聘,1.2-2万/月,07-10
248,seo优化,https://jobs.51job.com/dalian-zsq/116420102.ht...,广州华钧软件科技有限公司,异地招聘,1.2-2万/月,07-10


In [9]:
data.to_csv('job51.csv')

In [10]:
driver.close()